# **Data Extraction using API**



*   First, we extract the movie titles that we are going to work with, using the csv file we already have.



*   Then, extract each movie's data from the provided API and put it in a Data Frame.


*   Finaly, we save our data locally to csv file.




In [52]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import ast
# Extracting the titles from the csv file
movies = pd.read_csv("/content/Movies title.csv")

# Store the titles in a list
titles = []
for i in movies['Title']:
  titles.append(i)

data = []

for i in titles:
    # Customize the URL with the movie name as a variable that changes
    # throughout the loop
    URL = "https://www.omdbapi.com/?t="+i+"&apikey=fe1607ca"

    response = requests.get(URL)

    soup = BeautifulSoup(response.content, "html.parser")

    # Reformat the data record from a string to a dictionary
    d = ast.literal_eval(soup.text)

    # Append the
    data.append(d)

df = pd.DataFrame.from_records(data)

# Data **Transformation**


*  Transform the movies dataset  to the appropriate shape referring to the schema.

*   First, we apply the basic transformation to make sure our data is clean and in the correct format.

     1.   Dealing with nulls  (dropping it).
     2.   Format revision (data types, charset, date and time format).


*   Then, we apply the advanced transformation to enhance the quality of our data.

    1.   Derivation (deriving new attributes).

    2.   Dropping the unwanted columns for analysis (as the business owner specified before).
    3. Splitting (this is a useful step for dealing with multivalued attributes, splitting them into multiple single valued rows).
    4. Summarization (this step is for you, It will help you gain more insights about the data so make a good usage of it)







In [53]:
# Droping the unwanted columns for analysis (as the business owner specified)
df.drop(['Poster', 'DVD', 'Type', 'Awards' , 'Ratings' , 'totalSeasons' ,
         'Response', 'Error' , 'Website' , 'Production'], axis=1, inplace=True)

# Dealing with null and empty cells
df = df.dropna()

# ---------  Clear Data -------------- #

# Changing the 'rumtime' column from string to int
# Remove 'min' from Runtime column
# Dealing with cells that have 'N/A' values
df['Runtime'] = df['Runtime'].str.replace('N/A','0')
df['Runtime'] = df['Runtime'].str.strip('min')

# Cast 'Metascore' and 'imdbRating' to int type
# Delete rows with empty string
df['Metascore'] = df['Metascore'].str.replace('N/A','')
df = df[df.Metascore != '']
df = df[df.imdbRating !='']



# Changing the 'imbdVotes' column from string to int
# Remove the ',' from the column
df['imdbVotes'] = df['imdbVotes'].str.replace(',','')


# Changing the 'Boxoffice' column from string to int
# Remove the ',' and '$' from the column
df['BoxOffice'] = df['BoxOffice'].str.replace(',','')
df['BoxOffice'] = df['BoxOffice'].str.replace('$','')
df['BoxOffice'] = df['BoxOffice'].str.replace('N/A','0')
df = df[df.BoxOffice != '0']

df['Runtime'] = df['Runtime'].astype(int)
df['BoxOffice'] = df['BoxOffice'].astype(int)
df['imdbRating'] = df['imdbRating'].astype(float)
df['Metascore'] = df['Metascore'].astype(int)

pd.set_option('display.max_rows', None)

# Save the data to a csv file
df.to_csv("movies_data.csv")

<ipython-input-53-3c603492aa3e>:32: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['BoxOffice'] = df['BoxOffice'].str.replace('$','')


In [54]:
# Splitting Language column (Since, it is a multivalued attribute)

# To append the languages of each row after spliting
List = []
# Map to connect language with an ID
language_id = {}
#ID counter
ID = 1

# Iterate over the dataframe
for i, row in df.iterrows():
    # Getting Film ID & splitting the language column
    movieID = row['imdbID']
    language = str(row['Language']).replace(" ", "").split(',')
    # Iterate through every language to make the list
    for l in language:
      # Check if the language already has an ID
        if l not in language_id:
          # Assign each with language an ID
            language_id[l] = ID
            ID += 1
        List.append({'MovieID': movieID, 'Language': l, 'LanguageID': language_id[l]})

# Create a new DataFrame with the film_languages table
film_language= pd.DataFrame(List)
# Remove Duplicates
film_language = film_language.drop_duplicates(subset = "LanguageID")
film_language.to_csv("film_language.csv")
film_language

,MovieID,Language,LanguageID
0,tt2015381,English,1
2,tt1446714,Gaelic,2
5,tt3470600,Japanese,3
8,tt1386697,Spanish,4
10,tt2034800,Mandarin,5
13,tt3783958,Cantonese,6
15,tt1212428,Portuguese,7
16,tt1212428,Tupi,8
18,tt1212428,German,9
22,tt1355644,French,10


In [55]:
# Splitting Actors Column

# List where the actors data will be saved
List = []
# Map to connect actor with an ID
actor_id = {}
# ID counter
ID = 1

# Iterate over the dataframe
for i, row in df.iterrows():
    # Getting Film ID & splitting the actors column
    movieID = row['imdbID']
    actor = str(row['Actors']).replace(" ", "").split(',')
    for a in actor:
      # Check if the actor already has an ID
        if a not in actor_id:
          # Assign the actor to an ID
            actor_id[a] = ID
            ID += 1
        List.append({'MovieID': movieID, 'Actor': a, 'ActorID': actor_id[a]}) #.....

# Create a new data frame with the film_actor table
film_actor = pd.DataFrame(List)
# Remove Duplicates
film_actor = film_actor.drop_duplicates(subset = "ActorID")
film_actor.to_csv("film_actor.csv")
film_actor

,MovieID,Actor,ActorID
0,tt2015381,ChrisPratt,1
1,tt2015381,VinDiesel,2
2,tt2015381,BradleyCooper,3
3,tt1446714,NoomiRapace,4
4,tt1446714,LoganMarshall-Green,5
5,tt1446714,MichaelFassbender,6
6,tt4972582,JamesMcAvoy,7
7,tt4972582,AnyaTaylor-Joy,8
8,tt4972582,HaleyLuRichardson,9
9,tt3470600,MatthewMcConaughey,10


In [56]:
# Splitting Writers column

List = []
writer_id = {}
ID = 1

# Iterate over the dataframe
for i, row in df.iterrows():
    # Getting Film ID & splitting the writer column
    movieID = row['imdbID']
    writer = str(row['Writer']).replace(" ", "").split(',') #..
    for w in writer:
        # Check if the writer already has an ID
        if w not in writer_id:
           # Assign the writer to an ID
            writer_id[w] = ID
            ID += 1
        List.append({'MoveiID': movieID, 'Writer': w, 'WriterID': writer_id[w]})

# Create a new data frame with the film_actor table
film_writer = pd.DataFrame(List)
# Remove Duplicates
film_writer = film_writer.drop_duplicates(subset = "WriterID")
film_writer.to_csv("film_writer.csv")
film_writer

,MoveiID,Writer,WriterID
0,tt2015381,JamesGunn,1
1,tt2015381,NicolePerlman,2
2,tt2015381,DanAbnett,3
3,tt1446714,JonSpaihts,4
4,tt1446714,DamonLindelof,5
5,tt1446714,DanO'Bannon,6
6,tt4972582,M.NightShyamalan,7
7,tt3470600,GarthJennings,8
8,tt1386697,DavidAyer,9
9,tt1386697,JohnOstrander,10


In [57]:
# Splitting Genres column

List = []
Genre_id = {}
ID = 1

# Iterate over the dataframe
for i, row in df.iterrows():
    movieID = row['imdbID']
    genre = str(row['Genre']).replace(" ", "").split(',')
    for g in genre:
        # Check if the Genre already has an ID
        if g not in Genre_id:
            # Assign the Genre to an ID
            Genre_id[g] = ID
            ID += 1
        List.append({'MovieID': movieID, 'Genre': g, 'GenreID': Genre_id[g]})

# Create a new data frame with the film_genre table
film_genre = pd.DataFrame(List)
# Remove Duplicates
film_genre = film_genre.drop_duplicates(subset = "GenreID")
film_genre.to_csv("film_genre.csv")
film_genre

,MovieID,Genre,GenreID
0,tt2015381,Action,1
1,tt2015381,Adventure,2
2,tt2015381,Comedy,3
4,tt1446714,Mystery,4
5,tt1446714,Sci-Fi,5
6,tt4972582,Horror,6
7,tt4972582,Thriller,7
8,tt3470600,Animation,8
10,tt3470600,Family,9
13,tt1386697,Fantasy,10


In [58]:
# Splitting Countries column

List = []
Country_id = {}
ID = 1

# Iterate over the dataframe
for i, row in df.iterrows():
    movieID = row['imdbID']
    contry = str(row['Country']).replace(" ", "").split(',')
    for c in contry:
        # Check if the Country already has an ID
        if c not in Country_id:
            # Assign the Genre to an ID
            Country_id[c] = ID
            ID += 1
        List.append({'MovieID': movieID, 'Country': c, 'CountryID': Country_id[c]})

# Create a new data frame with the film_country table
film_country = pd.DataFrame(List)
# Remove Duplicates
film_country = film_country.drop_duplicates(subset = "CountryID")
film_country.to_csv("film_country.csv")
film_country

,MovieID,Country,CountryID
0,tt2015381,UnitedStates,1
1,tt1446714,UnitedKingdom,2
4,tt4972582,Japan,3
7,tt3470600,France,4
10,tt2034800,China,5
11,tt2034800,HongKong,6
12,tt2034800,Australia,7
13,tt2034800,Canada,8
25,tt4680182,Spain,9
27,tt4680182,SouthKorea,10


In [59]:
# Splitting Countries column

List = []
Country_id = {}
ID = 1

# Iterate over the dataframe
for i, row in df.iterrows():
    movieID = row['imdbID']
    contry = str(row['Country']).replace(" ", "").split(',')
    for c in contry:
        # Check if the Country already has an ID
        if c not in Country_id:
            # Assign the Genre to an ID
            Country_id[c] = ID
            ID += 1
        List.append({'MovieID': movieID, 'Country': c, 'CountryID': Country_id[c]})

# Create a new data frame with the film_country table
film_country = pd.DataFrame(List)
# Remove Duplicates
film_country = film_country.drop_duplicates(subset = "CountryID")
film_country.to_csv("film_country.csv")
film_country

,MovieID,Country,CountryID
0,tt2015381,UnitedStates,1
1,tt1446714,UnitedKingdom,2
4,tt4972582,Japan,3
7,tt3470600,France,4
10,tt2034800,China,5
11,tt2034800,HongKong,6
12,tt2034800,Australia,7
13,tt2034800,Canada,8
25,tt4680182,Spain,9
27,tt4680182,SouthKorea,10


In [62]:
import numpy as np
# Splitting Released Date column

List = []
for i, row in df.iterrows():
  imdbID = row['imdbID']
  date = str(row['Released'])
  day, month, year = date.split()
  List.append({"Day":day,"Month":month,"Year":year,"imdbID":imdbID})
rd_ID = np.arange(len(df.index))
released_date = pd.DataFrame(List)
released_date["ReleasedDateID"] = rd_ID +1
released_date.to_csv("released_date.csv")
released_date

,Day,Month,Year,imdbID,ReleasedDateID
0,01,Aug,2014,tt2015381,1
1,08,Jun,2012,tt1446714,2
2,20,Jan,2017,tt4972582,3
3,21,Dec,2016,tt3470600,4
4,05,Aug,2016,tt1386697,5
5,17,Feb,2017,tt2034800,6
6,25,Dec,2016,tt3783958,7
7,21,Apr,2017,tt1212428,8
8,21,Dec,2016,tt1355644,9
9,18,Nov,2016,tt3183660,10
